In [124]:
from psychopy import data, visual, core, event, monitors # import some libraries from PsychoPy
import numpy as np
import psychopy.tools.monitorunittools as unittools
import pandas as pd
import time

In [126]:
# Subject Settings
subID = 'PRACTICE'
numbersOdd = 1
blockID = 1
session = 1

if session == 1:
    prev_filename = []
elif session == 2:
    if blockID == 5:
        prev_filename = 'S0-JL-B1-Space-20191021-003808.pkl'
    elif blockID == 6:
        prev_filename = 'S0-JL-B2-Number-20191021-003842.pkl'
    elif blockID == 7:
        prev_filename = 'S0-JL-B3-Space-20191021-003903.pkl'
    elif blockID == 8:
        prev_filename = 'S0-JL-B4-Number-20191021-003938.pkl'
        
# Stim Settings
ntrials    = 10
Ns         = [2,3,4,5,6]
rangemax    = 100

#If session 1, leave blank!
#If session 2, B5 from B1
#              B6 from B2
#              B7 from B3
#              B8 from B4

if blockID==1 or blockID==3 or blockID==6 or blockID==8:
    oddblock = 1
else:
    oddblock = 0
        
if (numbersOdd and oddblock) or (not numbersOdd and not oddblock):
    print('number')
    numblock = True
else:
    print('space')
    numblock = False
    
if session == 1:
    X, N = config_x(Ns, ntrials)
elif session == 2:
    X, N = config_x(Ns, ntrials, prev_filename)

# Randomize vectors together
randomize = np.arange(ntrials)
np.random.shuffle(randomize)

N      = N[randomize]
X      = [X[i] for i in randomize]

# Monitor settings
mon = config_monitor()

# Screen settings
screenwidth = 1440
screenheight = 900

# Scale settings
scalewidth = 800

bg_colour = [0,0,0]
win = visual.Window(
    size=[screenwidth, screenheight],
    units="pix",
    fullscr=True,
    color=bg_colour,
    colorSpace='rgb',
    mouseVisible = True,
    monitor = mon
)

# Clock settings
timer = core.Clock()

# Line settings
line = visual.Line(
    win=win,
    units="pix",
    lineColor=[-1, -1, -1]
)

# run block
if numblock:
    blockname = 'Number'
elif not numblock:
    blockname = 'Space'
    
# Create dictionary
data = {'Num_block':[], #as opposed to space block (True/false)
        'Block_ID':[],
        'Trial_ID':[],
        'Resp_loc':[], 
        'Resp_conf':[], 
        'Correct':[],
        'RT_resp':[], 
        'RT_conf':[],
        'Wager':[],
        'X':[],
        'N':[],
        'x_mean':[],
        'Mouse_track':[]}

# Create dataframe
df = pd.DataFrame(data)


for trial in range(ntrials):    

    #rect_locs, rect_width, RTs = get_rect_response(np.around(x[trial][0]), rect_locs, rect_widths, RTs)
    resp_loc, rt_resp, mouse_track, escpressed = get_point_response(X[trial][0])
    
    if escpressed:
        timestr = time.strftime("%Y%m%d-%H%M%S")
        filename_csv = str('INCOMPLETE_S' + subID + '-B' + str(blockID) + '-' + blockname + '-' + timestr + '.csv')
        filename_pickle = str('INCOMPLETE_S' + subID + '-B' + str(blockID) + '-' + blockname + '-' + timestr + '.pkl')
        print(filename_pickle)
        df.to_csv(filename_csv)
        df.to_pickle(filename_pickle)
        
        win.close()
        core.quit()

    resp_conf, rt_conf, wager, correct = get_conf_response(X[trial][0],resp_loc)

    currtrial = pd.DataFrame({'Num_block':[numblock],
                              'Block_ID':[blockID],
                              'Trial_ID':[trial+1],
                              'Resp_loc':[resp_loc],
                              'Resp_conf':[resp_conf],
                              'Correct':[correct],
                              'RT_resp':[rt_resp],
                              'RT_conf':[rt_conf],
                              'Wager':[wager],
                              'X': [X[trial]],
                              'N': [N[trial]], 
                              'x_mean': [np.mean(X[trial])],
                              'Mouse_track':[mouse_track]})
    df = df.append(currtrial,ignore_index=True,sort=False)

timestr = time.strftime("%Y%m%d-%H%M%S")
filename_pickle = str('S' + subID + '-B' + str(blockID) + '-' + blockname + '-' + timestr + '.pkl')
filename_csv = str('S' + subID + '-B' + str(blockID) + '-' + blockname + '-' + timestr + '.csv')
print(filename_pickle)
df.to_pickle(filename_pickle)
df.to_csv(filename_csv)

win.close()
core.quit()

number
using new X stimuli
INCOMPLETE_SPRACTICE-B1-Number-20191021-113016.pkl


SystemExit: 0

/anaconda2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [125]:
def config_x(Ns, ntrials, *varg):
    if len(varg)>0:
        print('using previous X stimuli')
        dfprevious = pd.read_pkl(str(varg[0]))
        X = dfprevious.X
        N = dfprevious.N
    elif len(varg)==0:
        print('using new X stimuli')
        
        # Seed stimuli
        N       = np.tile(Ns, int(ntrials/len(Ns)))

        # Get stimuli x-values
        X = list()
        for i_trial in range(ntrials):
            X.append(np.random.uniform(0,100,[1,N[i_trial]]))
            
    return X, N

def config_monitor():
    my_monitor = monitors.Monitor(name='mac_monitor')
    my_monitor.setSizePix((1280, 800))
    my_monitor.setWidth(20)
    my_monitor.setDistance(100)
    my_monitor.saveMon()
    return my_monitor

def config_rect_cursor(xpos, width):
    rect = visual.Rect(
        win=win,
        units="pix",
        width=width,
        height=30,
        fillColor=[1, -1, -1],
        opacity=0.3,
        pos = [x2pix(xpos),0]
    )
    return rect

def get_point_response(x_):
    myMouse = event.Mouse(visible = True, win = win)
    mouse_down_detected = False
    timer.reset()
    mouse_track = []
    
    if numblock:
        halfheight = 100
    elif not numblock:
        halfheight = 200
        
    while not mouse_down_detected:
        keys = event.getKeys()
        if keys:
            if keys[0] == 'escape':
                resp_loc = None
                rt = None
                mouse_track = None
                escpressed = 1
                return resp_loc, rt, mouse_track, escpressed
    
        mouse_loc_pix = myMouse.getPos()
        xpos = pix2x(mouse_loc_pix[0])
        mouse_click = myMouse.getPressed()
        
        if xpos<0:
            xpos = 0
        elif xpos>100:
            xpos = 100
        
        if numblock:
            plot_numstim(x_)
        elif not numblock:
            plot_linestim(x_)
            
        disp_numberline()
        
        mouse_track.append(xpos) # Mouse-track xpos
        
        if mouse_click[0]: # button 0 is pressed
            if not mouse_down_detected: # initial detection
                rt = timer.getTime()
                mouse_down_detected = True
                if numblock:
                    resp_loc = np.round(xpos)
                elif not numblock:
                    resp_loc = xpos
                                            
        else: # button 0 is NOT pressed
            mouse_down_detected = False
            line = config_line(xpos,(1.0,-1,-1),halfheight)
            line.draw()
            if numblock:
                draw_dispnum(xpos,(x2pix(xpos)-12,-170),20)
            
            win.flip()
    
    mouse_track = np.array([mouse_track])
    
    escpressed = 0
    return resp_loc, rt, mouse_track, escpressed

def get_conf_response(x_,resp_loc):
    myMouse = event.Mouse(visible = True, win = win)
    space_down_detected = False
    timer.reset()
    if numblock:
        halfheight = 100
    elif not numblock:
        halfheight = 200

    while not space_down_detected:
        mouse_loc_pix = myMouse.getPos()
        resp_conf_ = abs(pix2x(mouse_loc_pix[0])-resp_loc)*2
        pts = 15*np.exp(-resp_conf_/20)
        mouse_click = myMouse.getPressed()
        
        if numblock:
            plot_numstim(x_)
        elif not numblock:
            plot_linestim(x_)
        disp_numberline()
        
        keys = event.getKeys(['space'])
        if keys != []:
            if keys[0] == 'space': # space bar is pressed
                if not space_down_detected: # initial detection
                    rt        = timer.getTime()
                    resp_conf = resp_conf_
                    wager     = round(int(pts))
                    space_down_detected = True
                rect=config_rect_cursor(resp_loc,resp_conf_*scalewidth/rangemax)
                rect.draw()
                
                # plot feedback
                line = config_line(np.mean(x_),[0, 1, 0],halfheight)
                line.draw()

                correct = (resp_loc - resp_conf/2) < np.mean(x_) < (resp_loc + resp_conf/2)

#                 print(rect_loc)
#                 print(rect_width)
#                 print(np.mean(x_))

                if correct:
                    fb2 = '+$' + str(wager)
                    fbcolor = (-1,1,-1)
                else:
                    wager = 0
                    fb2 = '+$0'
                    fbcolor = (1,-1,-1)

                #draw_fb(fb1, (0,0,1), (x2pix(np.mean(x_))-10,60))
                draw_fb(fb2, fbcolor, (-10,-300),40)
                disp_numberline()
                win.flip() 
                core.wait(1)

        else: # space bar is NOT pressed
            space_down_detected = False
            line = config_line(resp_loc,(1.0,-1,-1),halfheight)
            line.draw()
            if numblock:
                draw_dispnum(resp_loc,(x2pix(resp_loc)-12,-170),20)
            
            rect=config_rect_cursor(resp_loc,resp_conf_*scalewidth/rangemax)
            rect.draw()
            pt_text = ' $'+ str(round(int(pts)))
            draw_fb(pt_text,(-1,1,-1),(-10,-300),40)
            
            win.flip()

    return resp_conf, rt, wager, correct

def plot_linestim(x_):
    # plot stimuli
    for offset in x_:
        line = config_line(offset,[-1,-1,-1],150)
        line.draw()

def get_rect_response(x_, rect_locs, rect_widths, RTs):
    mouse_down_detected = False
    rect_width_pix = 10
    timer.reset()
    
    while not mouse_down_detected:
        mouse_loc_pix = unittools.deg2pix(myMouse.getPos(),mon)
        rect_loc = mouse_loc_pix[0]*(rangemax/screenwidth)+(rangemax/2)
        rect_width = rect_width_pix*(rangemax/screenwidth)
        mouse_click = myMouse.getPressed()
        if mouse_click[0]: # button 0 is pressed
            RT = timer.getTime()
            RTs.append(RT)
            
            if not mouse_down_detected: # initial detection
                rect_locs.append(rect_loc)
                rect_widths.append(rect_width)
                mouse_down_detected = True
            
            rect=config_rect_cursor(mouse_loc_pix[0],rect_width_pix)
            #cross=config_cross_cursor(myMouse.getPos()[0])
            rect.draw()
            #cross.draw()
            
            # plot stimuli
            draw_linestim(x_)
            
            # plot feedback
            line = config_line(np.mean(x_),[-1,1,-1],200)
            line.draw()
            
            correct = (rect_loc - rect_width/2) < np.mean(x_) < (rect_loc + rect_width/2)
            
            print(rect_loc)
            print(rect_width)
            print(np.mean(x_))
            
            if correct:
                pts = round(100/rect_width)
                fb = 'Correct    + ' + str(pts)
            else:
                pts = 0
                fb = 'Incorrect    + ' + str(pts)
                          
            draw_fb(fb, (-1,-1,-1), (0,0.75))
            #win.flip()
            #core.wait(1)
            
        else: # button 0 is NOT pressed
            mouse_down_detected = False
            
            keys = event.getKeys(['e','d'])
            if keys != []:
                if keys[0] == 'e':
                    rect_width_pix = rect_width_pix + 25
                elif keys[0] == 'd' and rect_width_pix > 10:
                    rect_width_pix = rect_width_pix - 25
                
            rect=config_rect_cursor(mouse_loc_pix[0],rect_width_pix)
            
            #cross=config_cross_cursor(myMouse.getPos()[0])
            rect.draw()
            #cross.draw()
            
            win.flip()
    return rect_locs, rect_widths, RTs

def config_line(xpos,colour,halfheight):
    line.start = [x2pix(xpos), -halfheight]
    line.end = [x2pix(xpos), +halfheight]
    line.setColor(colour,'rgb')
    #line.lineColorSpace='rgb'
    return line

def draw_fb(text, color, pos, height):
    text = visual.TextStim(win=win, text=text, color=color, colorSpace='rgb',units='pix', alignHoriz='left', alignVert='center', pos=pos, height=height)
    text.draw()

def x2pix(xx):
    pix = xx*scalewidth/rangemax - scalewidth/2
    return pix

def pix2x(pix):
    xx = (pix+scalewidth/2)*rangemax/scalewidth
    return xx  

def draw_dispnum(dispnum,posi,height):
    num = visual.TextStim(
                    win=win,
                    wrapWidth=350,
                    text=str(int(np.round(dispnum))),
                    pos=posi,
                    alignHoriz='left',
                    units="pix",
                    height = height,
                    color = (1,1,1)
                )
    num.draw()
    #return num

def disp_numberline():
    if not numblock:
        ticks = [0,100]
    elif numblock:
        ticks = [0,25,50,75,100]
        
    for ii in ticks:
        line.start = [x2pix(ii), -30]
        line.end = [(x2pix(ii)), +30]
        line.setColor((-1, -1, 1),'rgb')
        line.draw()
        if numblock:
            draw_dispnum(ii,(x2pix(ii)-10,+30),12)
    
    #horizontal line
    line.start = [x2pix(0),0]
    line.end = [x2pix(100),0]
    line.setColor((-1, -1, 1),'rgb')
    line.draw()
    
def plot_numstim(x_):
    sloth = 150
    slotw = 150
    margh = (screenheight - sloth*5)/2
    margw = (screenwidth - (len(x_)-1)*slotw - len(x_)*7) /2
    
    ipos = 1
    
    for number in x_:
        num = visual.TextStim(
            win=win,
            wrapWidth=350,
            alignHoriz = 'left',
            height=30,
            color = (-1,-1,-1)
        )
        num.text = str(int(number))

        row = np.floor((ipos-1)/6) + 1
        col = ipos-1#-(row-1)*6

        ypos = -((margh + row*sloth)-screenheight/2)
        xpos = (margw + col*slotw)-screenwidth/2

        num.pos = (xpos,ypos)

        num.draw()

        ipos = ipos + 1
